In [9]:
import cv2
import numpy as np
import mediapipe as mp

In [10]:
def list_available_cameras(max_index):
    """
    Lists the available camera indices.

    Args:
        max_index (int): The maximum index to check for available cameras.

    Returns:
        list: A list of indices where cameras are available.
    """

    # Store the indices of available cameras
    available_cameras = []

    # Iterate through the indices
    for i in range(max_index):
        cap = cv2.VideoCapture(i)

        # Check if the camera is available
        if cap.isOpened():
            # Add the index to the list of available cameras
            available_cameras.append(i)
            cap.release()        

    return available_cameras

max_camera_index = 10 # Maximum index to check for available cameras
available_cameras = list_available_cameras(max_camera_index) # Call the function

[ WARN:0@924.497] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ERROR:0@924.595] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@924.683] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video3): can't open camera by index
[ERROR:0@924.685] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@924.685] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video4): can't open camera by index
[ERROR:0@924.686] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@924.687] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video5): can't open camera by index
[ERROR:0@924.688] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@924.688] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video6): can't open camera by index
[ERROR:0@924.690] global obsensor_uvc_stream_channel.cpp:158 get

In [11]:
# Print the available cameras

print("Available Cameras:", available_cameras)

Available Cameras: [0, 2]


# Mathematical Explanation of classify_pose Function

The classify_pose function determines the pose based on the y-coordinates of specific body landmarks detected by MediaPipe's pose estimation model. These landmarks include points like the shoulders, wrists, and hips. The function uses the y-coordinates of these landmarks to classify the pose into three categories: `Paper`, `Rock,` or `Scissors`

## Pose Classification Logic

### 1. Paper Pose

**Condition:** Both wrists are higher (i.e., have a smaller y-coordinate) than both shoulders.

**Mathematical Representation:**

$$
y_{\text{left\_wrist}} < y_{\text{left\_shoulder}} \quad \text{and} \quad y_{\text{right\_wrist}} < y_{\text{right\_shoulder}}
$$

This means both wrists are raised above the shoulders, resembling a "Paper" gesture in rock-paper-scissors.

### 2. Rock Pose

**Condition:** Both wrists are lower (i.e., have a larger y-coordinate) than both hips.

**Mathematical Representation:**

$$
y_{\text{left\_wrist}} > y_{\text{left\_hip}} \quad \text{and} \quad y_{\text{right\_wrist}} > y_{\text{right\_hip}}
$$

This means both wrists are down by the sides or below the hips, resembling a "Rock" gesture in rock-paper-scissors.

### 3. Scissors Pose

**Condition:** If neither of the above conditions are met, it is classified as "Scissors."

This is the default classification when the wrists are not in positions that satisfy the conditions for "Paper" or "Rock."

## Example Breakdown

Let's consider an example with hypothetical y-coordinates for the left and right wrists, shoulders, and hips:

- $ y_{\text{left\_wrist}} = 0.4 $
- $ y_{\text{right\_wrist}} = 0.35 $
- $ y_{\text{left\_shoulder}} = 0.5 $
- $ y_{\text{right\_shoulder}} = 0.55 $
- $ y_{\text{left\_hip}} = 0.6 $
- $ y_{\text{right\_hip}} = 0.65 $

### Classification Steps:

1. **Check for Paper:**

$$
y_{\text{left\_wrist}} (0.4) < y_{\text{left\_shoulder}} (0.5) \quad \text{and} \quad y_{\text{right\_wrist}} (0.35) < y_{\text{right\_shoulder}} (0.55)
$$

Both conditions are true, so the pose is classified as "Paper."

2. **Check for Rock:**

This step is skipped because the "Paper" condition is already satisfied.

3. **Check for Scissors:**

This step is skipped because one of the previous conditions is already satisfied.

## Summary

The classification involves comparing y-coordinates of specific body landmarks:

- For "Paper": Both wrists must be above the shoulders.
- For "Rock": Both wrists must be below the hips.
- For "Scissors": Any other configuration that does not meet the above criteria.

This approach translates the spatial positions of the wrists relative to other body parts into specific gestures used in rock-paper-scissors.

In [12]:
def classify_pose(landmarks):
    """
    Classifies the pose based on the position of the wrist landmarks relative to the shoulder and hip landmarks.

    Args:
        landmarks (list): A list of pose landmarks.

    Returns:
        str: The classified pose ('Paper', 'Rock', or 'Scissors').
    """
    
    if landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y < landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y and \
       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].y < landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y:
        return "Paper"
    elif landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y > landmarks[mp_pose.PoseLandmark.LEFT_HIP].y and \
         landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].y > landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y:
        return "Rock"
    else:
        return "Scissors"

In [13]:
# Initialize MediaPipe pose solution
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize the video capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame and get pose landmarks
    result = pose.process(frame_rgb)

    if result.pose_landmarks:
        # Draw pose landmarks
        mp.solutions.drawing_utils.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Classify the pose
        pose_class = classify_pose(result.pose_landmarks.landmark)
        cv2.putText(frame, pose_class, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow('Rock-Paper-Scissors', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open zink: /usr/lib/dri/zink_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: 